# Nhận dạng thực thể có tên sử dụng sklearn-crfsuite

Huấn luyện mô hình CRF cơ bản cho nhận dạng thực thể có tên trên dữ liệu CoNLL2002 và kiểm tra trọng số của mô hình để xem mô hình học được những gì.

Các thư viện Python NLTK> 3.x và sklearn-crfsuite, sử dụng Python 3.




In [1]:
# !pip install sklearn_crfsuite

In [2]:
# !pip install eli5

In [3]:
#from nltk.corpus import conll2002
#nltk.download('conll2002')

In [4]:
import numpy as np
import nltk
import sklearn_crfsuite
import eli5

# 1. Dữ liệu huấn luyện (Training data)

Bộ dữ liệu CoNLL 2002 chứa danh sách các câu tiếng Tây Ban Nha, với các thực thể có tên được chú giải. Bộ dữ liệu này sử dụng mã hóa IOB2. Dữ liệu CoNLL 2002 cũng cung cấp các thẻ POS.

In [5]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

# 2. Trích xuất đặc trưng

Thẻ POS có thể được xem như là các đặc trưng được trích xuất trước. Hãy trích xuất các đặc trưng khác (word parts, thẻ từ loại đã rút gọn - simplified POS tags, lower/title/upper flags, đặc trưng của các từ lân cận - features of nearby words) và chuyển đổi chúng sang định dạng sklear-crfsuite - mỗi một câu phải được chuyển đổi thành một danh sách của từ điển.

In [6]:
def word2features(sent, i):

    # Lấy từ và nhãn tương ứng
    # sent là 1 câu, i là từ thứ i trong câu, còn index 0, 1  là để lấy word và postag
    # nếu là [i][2] thì lấy postag BIO
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True # BOS: Begin Of Sentence

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True # EOS: End Of Sentence

    return features


# Return về 1 list với mỗi phần tử là 1 kết quả trả về của việc gọi word2features()
# Duyệt từng từ trong câu, mỗi từ trích xuất đặc trưng
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train_sents]
Y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
Y_test = [sent2labels(s) for s in test_sents]


Các đặc trưng được trích xuất từ một token duy nhất có dạng như sau:

In [7]:
X_train[0][1] # câu 0, từ 1

{'bias': 1.0,
 'word.lower()': '(',
 'word[-3:]': '(',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'Fpa',
 'postag[:2]': 'Fp',
 '-1:word.lower()': 'melbourne',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:postag': 'NP',
 '-1:postag[:2]': 'NP',
 '+1:word.lower()': 'australia',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False,
 '+1:postag': 'NP',
 '+1:postag[:2]': 'NP'}

# 3. Huấn luyện mô hình CRF

Khi có các đặc trưng ở định dạng phù hợp ta có thể huấn luyện mô hình CRF chuỗi tuyến tính (Conditional Random Field - CRFs) bằng cách sử dụng sklearn_crfsuite.CRF:

In [8]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', # Limited-memory Broyden–Fletcher–Goldfarb–Shanno, gradient descent
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)

crf.fit(X_train, Y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    max_iterations=20)

# 4. Kiểm tra trọng số mô hình

Các mô hình CRF của CRFsuite sử dụng hai loại đặc trưng: đặc trưng trạng thái và đặc trưng chuyển tiếp. Hãy kiểm tra trọng số của chúng bằng eli5.explain_weights:



In [9]:
eli5.show_weights(crf, top=30)


Các đặc trưng chuyển tiếp có ý nghĩa: ít nhất mô hình đã học rằng I-ENTITY phải theo sau B-ENTITY. Nó cũng học được rằng một số chuyển tiếp khó xảy ra, ví dụ: ngay sau tên tổ chức thường không phải là một vị trí (I-ORG -> B-LOC có trọng số âm lớn).

Các đặc trưng không sử dụng từ điển địa lý, vì vậy mô hình phải nhớ một số tên địa lý từ dữ liệu huấn luyện, ví dụ: España là một địa điểm.

Nếu chúng ta regularize CRF hơn, chúng ta có thể kỳ vọng rằng các đặc trưng duy nhất mang tính tổng quát sẽ vẫn còn, và thẻ ghi nhớ sẽ mất đi. Với hệ số chính quy L1 (tham số c1) của hầu hết các đặc trưng sẽ được chuyển về không. Hãy kiểm tra xem việc regularization có ảnh hưởng gì đến trọng số CRF:


In [10]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=200,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, Y_train)
eli5.show_weights(crf, top=30)

Như chúng ta có thể thấy, các thẻ được ghi nhớ hầu hết đã biến mất và mô hình hiện dựa vào khuôn dạng từ (word shapes) và nhãn từ loại (POS tags). Chỉ còn lại một số đặc trưng khác không. Trong ví dụ này, sự thay đổi có thể khiến chất lượng kém hơn, nhưng đó là một vấn đề riêng.

Hãy tập trung vào trọng số chuyển tiếp. Chúng ta có thể cho rằng các chuyển tiếp O -> I-ENTIRY có trọng số âm lớn vì chúng là không thể. Nhưng những chuyển tiếp này có trọng số bằng không, không có trọng số âm, cả trong mô hình được chính quy hóa nhiều và trong mô hình ban đầu. Có gì đó đang xảy ra ở đây.

Lý do chúng bằng 0 là crfsuite đã không nhìn thấy những chuyển đổi này trong dữ liệu huấn luyện và để tiết kiệm thời gian tính toán nên đã cho rằng không cần phải học trọng số đối với chúng. Đây là mặc định, nhưng có thể tắt nó bằng cách sử dụng sklearn_crfsuite.CRF tùy chọn all_possible_transitions. Hãy kiểm tra xem nó ảnh hưởng như thế nào đến kết quả:

In [11]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=True,
)
crf.fit(X_train, Y_train)

#Hiển thị trọng số với top 5 dùng transition_features
eli5.show_weights(crf, top=5)

Với all_possible_transitions = True CRF đã học được các trọng số âm lớn cho các chuyển đổi không thể như O -> I-ORG.

# 5. Tùy chỉnh
Bảng trên lớn và khó quan sát, eli5 cung cấp một số tùy chọn để chỉ xem xét một phần của các đặc trưng. Chúng ta có thể chỉ kiểm tra một tập hợp con các nhãn:

In [12]:
eli5.show_weights(crf, top=10, targets=['O', 'B-ORG', 'I-ORG'])

Một tùy chọn khác là chỉ kiểm tra một số đặc trưng - nó giúp kiểm tra xem một hàm đặc trưng có hoạt động như dự kiến hay không. Ví dụ: hãy kiểm tra cách mô hình sử dụng các đặc trưng khuôn dạng từ bằng cách sử dụng đối số feature_re và ẩn bảng chuyển tiếp:

In [13]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\congh\AppData\Local\Temp\ipykernel_1604\672144811.py:1: SyntaxWarning: invalid escape sequence '\.'
  eli5.show_weights(crf, top=10, feature_re='^word\.is',


Weight?,Feature
-3.685,word.isupper()
-7.025,word.istitle()
Weight?,Feature
+2.397,word.istitle()
+0.099,word.isupper()
-0.152,word.isdigit()
Weight?,Feature
+0.460,word.istitle()
-0.018,word.isdigit()
-0.345,word.isupper()


# 6. Trả kết quả về dạng văn bản
Cũng có thể định dạng kết quả dưới dạng văn bản (có thể xuất ra console):

In [14]:
expl = eli5.explain_weights(crf, top=5, targets=['O', 'B-LOC', 'I-LOC'])
print(eli5.format_as_text(expl))

Explained as: CRF

Transition features:
            O    B-LOC    I-LOC
-----  ------  -------  -------
O       2.732    1.217   -4.675
B-LOC  -0.226   -0.091    3.378
I-LOC  -0.184   -0.585    2.404

y='O' top features
Weight  Feature       
------  --------------
+4.931  BOS           
+3.754  postag[:2]:Fp 
+3.539  bias          
… 15043 more positive …
… 3906 more negative …
-3.685  word.isupper()
-7.025  word.istitle()

y='B-LOC' top features
Weight  Feature           
------  ------------------
+2.397  word.istitle()    
+2.147  -1:word.lower():en
  … 2284 more positive …  
  … 433 more negative …   
-1.080  postag:SP         
-1.080  postag[:2]:SP     
-1.273  -1:word.istitle() 

y='I-LOC' top features
Weight  Feature           
------  ------------------
+0.882  -1:word.lower():de
+0.780  -1:word.istitle() 
+0.718  word[-3:]:de      
+0.711  word.lower():de   
  … 1684 more positive …  
  … 268 more negative …   
-1.965  BOS               

